In [ ]:
import sqlite3
import pandas as pd
from scipy.spatial import cKDTree
import math

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 

In [ ]:
fn = 'points.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))

# activity

In [ ]:
import requests
import pandas as pd

overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25];
nwr["highway"="hitchhiking"];
out geom;
"""

response = requests.post(overpass_url, data={'data': overpass_query})
data = response.json()

# Extract nodes
elements = data['elements']
nodes = [el for el in elements if el['type'] == 'node']

# Convert to DataFrame for easier handling
nodes_df = pd.DataFrame([{
    'id': node['id'],
    'lat': node['lat'],
    'lon': node['lon'],
    'tags': node.get('tags', {})
} for node in nodes])

len(nodes_df)

In [ ]:
tree = cKDTree(points[['lat', 'lon']].values)

distances, indices = tree.query(nodes_df[['lat', 'lon']].values)

# Add nearest node info to points DataFrame
nodes_df['nearest_node_id'] = points.iloc[indices]['id'].values
nodes_df['nearest_node_lat'] = points.iloc[indices]['lat'].values
nodes_df['nearest_node_lon'] = points.iloc[indices]['lon'].values
nodes_df['distance'] = distances

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

In [ ]:
nodes_df['haversine_distance'] = nodes_df.apply(lambda row: haversine(row['lat'], row['lon'], row['nearest_node_lat'], row['nearest_node_lon']) * 1000, axis=1)

In [ ]:
nodes_df = nodes_df.sort_values(by='haversine_distance')

In [ ]:
nodes_df.head(100)